<a href="https://colab.research.google.com/github/ANAVDUTT/IPHS290_CulturalAnalytics/blob/main/Copy_of_another_one.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# REST URL ONLY: Scrape Twitter for US Governors in 2022 Mid-Terms

# Setup

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import glob

import requests

import json
from pprint import pprint

In [ ]:
import tweepy
import gc #collecting garbage
from wordcloud import WordCloud,STOPWORDS, ImageColorGenerator #visualisation

sns.set_theme(context='notebook', style='whitegrid', palette='vlag', font='serif')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Get Data

In [ ]:
govs_df = pd.read_csv('twitter_2022midterm_us_governors.csv', index_col=[0])
govs_df.head()
govs_df.info()

,Republican,Democrat
State,,
Wisconsin:,https://twitter.com/michelsforgov,https://twitter.com/GovEvers
Orgeon:,https://twitter.com/ChristineDrazan,https://twitter.com/TinaKotek
Nevada:,https://twitter.com/JoeLombardoNV,https://twitter.com/GovSisolak
Arizona:,https://twitter.com/KariLake,https://twitter.com/katiehobbs
Kansas:,https://twitter.com/DerekSchmidtKS,https://twitter.com/LauraKellyKS


<class 'pandas.core.frame.DataFrame'>
Index: 35 entries, Wisconsin: to Idaho:
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Republican  35 non-null     object
 1   Democrat    35 non-null     object
dtypes: object(2)
memory usage: 840.0+ bytes


In [ ]:
# Clean imported DataFrame

# Lowercase Columns
govs_df.rename(columns={'Republican':'republican', 'Democrat':'democrat'}, inplace=True)


# Move State Index to separate colum for joining
state_orig_ls = govs_df.index.to_list()
# state_orig_ls[:5]

state_clean_ls = [x.lower().strip(':') for x in state_orig_ls]
# state_clean_ls[:5]

# Insert state col in first position
govs_df.insert(loc=0, column='state', value=state_clean_ls)
# govs_df['state'] = state_clean_ls

# Drop the State Index
govs_df.reset_index(drop=True, inplace=True)

govs_df.head()


,state,republican,democrat
0,wisconsin,https://twitter.com/michelsforgov,https://twitter.com/GovEvers
1,orgeon,https://twitter.com/ChristineDrazan,https://twitter.com/TinaKotek
2,nevada,https://twitter.com/JoeLombardoNV,https://twitter.com/GovSisolak
3,arizona,https://twitter.com/KariLake,https://twitter.com/katiehobbs
4,kansas,https://twitter.com/DerekSchmidtKS,https://twitter.com/LauraKellyKS


In [ ]:
# Split into Two (1) Dem and (2) Rep tables

govs_d_df = govs_df[['state','democrat']].copy(deep=True)
# Extract username from url
govs_d_df['screen_name'] = govs_df["democrat"].apply(lambda x: str(x).split('/')[-1])
govs_d_df.head()

govs_r_df = govs_df[['state','republican']].copy(deep=True) 
# Extract username from url
govs_r_df['screen_name'] = govs_df["republican"].apply(lambda x: str(x).split('/')[-1])
govs_r_df.head()

,state,democrat,screen_name
0,wisconsin,https://twitter.com/GovEvers,GovEvers
1,orgeon,https://twitter.com/TinaKotek,TinaKotek
2,nevada,https://twitter.com/GovSisolak,GovSisolak
3,arizona,https://twitter.com/katiehobbs,katiehobbs
4,kansas,https://twitter.com/LauraKellyKS,LauraKellyKS


,state,republican,screen_name
0,wisconsin,https://twitter.com/michelsforgov,michelsforgov
1,orgeon,https://twitter.com/ChristineDrazan,ChristineDrazan
2,nevada,https://twitter.com/JoeLombardoNV,JoeLombardoNV
3,arizona,https://twitter.com/KariLake,KariLake
4,kansas,https://twitter.com/DerekSchmidtKS,DerekSchmidtKS


# Get OAuth Tokens

In [ ]:
#import config_twitter

#api_key = config_twitter.API_KEY #Your API/Consumer key 
#api_secret = config_twitter.API_KEY_SECRET #Your API/Consumer Secret Key
bearer_token = ('AAAAAAAAAAAAAAAAAAAAALOHIwEAAAAA0BF5eAI0f%2Fjm%2FbGXTnbW%2FId6fkA%3DtAMiBtb4K1uyXXgj9UAgCy4hNMkA447ttQKgLaz2ncSLrTfiZv')
#access_token = config_twitter.ACCESS_TOKEN    #Your Access token key
#access_token_secret = config_twitter.ACCESS_TOKEN_SECRET #Your Access token Secret key

# REST URI Endpoints (v2.0 API)

Regular 

* https://github.com/twitterdev/Twitter-API-v2-sample-code

Academic

* https://github.com/twitterdev/getting-started-with-the-twitter-api-v2-for-academic-research/blob/main/modules/3-deciding-which-endpoints-to-use.md

In [ ]:
%whos DataFrame

Variable    Type         Data/Info
----------------------------------
govs_d_df   DataFrame                 state       <...>dtforgov      heidtforgov
govs_df     DataFrame                 state       <...>twitter.com/heidtforgov  
govs_r_df   DataFrame                 state       <...>alABundy       RealABundy


In [ ]:
# ONLY RUN if you skipped API v1/v2 search and merge above

govs_d_user_df = govs_d_df.copy()
govs_r_user_df = govs_r_df.copy()

In [ ]:
govs_d_user_df.head()
govs_d_user_df.info()

,state,democrat,screen_name
0,wisconsin,https://twitter.com/GovEvers,GovEvers
1,orgeon,https://twitter.com/TinaKotek,TinaKotek
2,nevada,https://twitter.com/GovSisolak,GovSisolak
3,arizona,https://twitter.com/katiehobbs,katiehobbs
4,kansas,https://twitter.com/LauraKellyKS,LauraKellyKS


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   state        35 non-null     object
 1   democrat     35 non-null     object
 2   screen_name  35 non-null     object
dtypes: object(3)
memory usage: 968.0+ bytes


## Common Functions

In [ ]:
def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2UserLookupPython"
    return r

In [ ]:
def connect_to_endpoint_wo_params(url):
    response = requests.request("GET", url, auth=bearer_oauth,)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Request returned an error: {} {}".format(
                response.status_code, response.text
            )
        )
    return response.json()

In [ ]:
def connect_to_endpoint_with_params(url, tweet_fields):
    response = requests.request(
        "GET", url, auth=bearer_oauth, params=tweet_fields)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Request returned an error: {} {}".format(
                response.status_code, response.text
            )
        )
    return response.json()

In [ ]:
def get_followers(user_id):
    url = "https://api.twitter.com/2/users/{}/following".format(user_id)
    params = {"user.fields": "created_at"}
    json_response = connect_to_endpoint_with_params(url, params)
    return json_response
    
# Test
user_id = 234896532
json_response = get_followers(user_id)
print(json.dumps(json_response, indent=4, sort_keys=True))

my_followers_df = pd.DataFrame(json_response['data'])
my_followers_df.head()
my_followers_df.info()
    

200
{
    "data": [
        {
            "created_at": "2021-05-04T19:31:07.000Z",
            "id": "1389663738945495040",
            "name": "Dr. Jill Underly, WI State Superintendent",
            "username": "DrJillUnderly"
        },
        {
            "created_at": "2021-01-06T17:02:07.000Z",
            "id": "1346864195304792065",
            "name": "Senator Brad Pfaff",
            "username": "SenBradPfaff"
        },
        {
            "created_at": "2009-05-27T19:35:11.000Z",
            "id": "42953875",
            "name": "Western Tech College",
            "username": "WesternTC"
        },
        {
            "created_at": "2008-07-21T18:48:42.000Z",
            "id": "15518969",
            "name": "Ducks Unlimited",
            "username": "DucksUnlimited"
        },
        {
            "created_at": "2018-05-01T14:15:25.000Z",
            "id": "991320189634478080",
            "name": "Forward Madison FC",
            "username": "ForwardMSNFC"
       

,created_at,username,name,id
0,2021-05-04T19:31:07.000Z,DrJillUnderly,"Dr. Jill Underly, WI State Superintendent",1389663738945495040
1,2021-01-06T17:02:07.000Z,SenBradPfaff,Senator Brad Pfaff,1346864195304792065
2,2009-05-27T19:35:11.000Z,WesternTC,Western Tech College,42953875
3,2008-07-21T18:48:42.000Z,DucksUnlimited,Ducks Unlimited,15518969
4,2018-05-01T14:15:25.000Z,ForwardMSNFC,Forward Madison FC,991320189634478080


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   created_at  100 non-null    object
 1   username    100 non-null    object
 2   name        100 non-null    object
 3   id          100 non-null    object
dtypes: object(4)
memory usage: 3.2+ KB


In [ ]:
def get_user_by_username(username, param_str):

    url = f'https://api.twitter.com/2/users/by/username/{username}?user.fields={param_str}'
    # url = f"https://api.twitter.com/2/users/by?user.fields={params}"
    params = {"user.fields": param_str}
    json_response = connect_to_endpoint_with_params(url, params)
    return json_response
    
# Test

username = 'KariLake'
params = 'created_at,description' # ,entities,id,location,name,pinned_tweet_id,profile_image_url,protected,public_metrics,url,username,verified,withheld'

user_id = 234896532
json_response = get_user_by_username(username, params)
print(json.dumps(json_response, indent=4, sort_keys=True))

400


Exception: ignored

In [ ]:
def get_user_by_username(username, param_str):

    url = f'https://api.twitter.com/2/users/by/username/{username}?user.fields={param_str}'
    # url = f"https://api.twitter.com/2/users/by?user.fields={params}"
    params = {"user.fields": param_str}
    json_response = connect_to_endpoint_wo_params(url) # , params)
    return json_response
    
# Test

username = 'KariLake'
params = 'created_at,description,entities,id,location,name,pinned_tweet_id,profile_image_url,protected,public_metrics,url,username,verified,withheld'

# user_id = 234896532
json_response = get_user_by_username(username, params)
print(json.dumps(json_response, indent=4, sort_keys=True))

my_followers_df = pd.DataFrame(pd.json_normalize(json_response['data']))
my_followers_df.head()
my_followers_df.info()



url = 'https://api.twitter.com/2/users/by?user.fields='
url


### User Info

In [ ]:
# Define DataFrame columns for all the User data elements we want to collect

govs_d_user_df = pd.DataFrame(columns=['screen_name',
                                    'id',
                                    'name',
                                    'location',
                                    'description',
                                    'url',
                                    'created_at',
                                    'verified',
                                    'protected',
                                    # 'followers',
                                    'followers_count',
                                    # 'followers_id',
                                    # 'following',
                                    'follow_request_sent',
                                    # 'follow',
                                    # 'friends',
                                    'friends_count',
                                    'favorites_count',
                                    'listed_count',
                                    # 'status',
                                    # 'statuses_count',
                                    ])

govs_r_user_df = pd.DataFrame(columns=['screen_name',
                                    'id',
                                    'name',
                                    'location',
                                    'description',
                                    'url',
                                    'created_at',
                                    'verified',
                                    'protected',
                                    # 'followers',
                                    'followers_count',
                                    # 'followers_id',
                                    # 'following',
                                    'follow_request_sent',
                                    # 'follow',
                                    # 'friends',
                                    'friends_count',
                                    'favorites_count',
                                    'listed_count',
                                    # 'status',
                                    # 'statuses_count',
                                    ])

In [ ]:
govs_d_df.head()

In [ ]:
username = 'KariLake'

# user_id = 234896532
json_response = get_user_by_username(username, params)
print(json.dumps(json_response, indent=4, sort_keys=True))



200
{
    "data": {
        "created_at": "2013-01-12T07:47:51.000Z",
        "description": "Official Twitter Account for Trump-Endorsed Candidate for Arizona Governor. Text KARI to 70789. Find me on TRUTH Social at karilake & FB at TheKariLake.",
        "entities": {
            "url": {
                "urls": [
                    {
                        "display_url": "KariLake.com",
                        "end": 23,
                        "expanded_url": "http://KariLake.com",
                        "start": 0,
                        "url": "https://t.co/EmzgORJKeQ"
                    }
                ]
            }
        },
        "id": "1082197856",
        "location": "Iowa & Arizona",
        "name": "Kari Lake",
        "pinned_tweet_id": "1575893306487103498",
        "profile_image_url": "https://pbs.twimg.com/profile_images/1560073130822885378/55oLUgB0_normal.jpg",
        "protected": false,
        "public_metrics": {
            "followers_count": 716331,


In [ ]:
# Get User data for each Dem Governor

govs_d_user_df = pd.DataFrame()

params = 'created_at,description,entities,id,location,name,pinned_tweet_id,profile_image_url,protected,public_metrics,url,username,verified,withheld'

i = 0
for i,agov_username in enumerate(govs_d_df.screen_name.to_list()):
    print(f'Processing #{i}: {agov_username}')
    try:
        json_response = get_user_by_username(agov_username, params)
        # print(json.dumps(json_response, indent=4, sort_keys=True))

        my_userdata_df = pd.DataFrame(pd.json_normalize(json_response['data']))
        
        print(f'    Appending row: {my_userdata_df.head()}\n\n')
        
        if i == 0:
            govs_d_user_df = my_userdata_df.copy(deep=True)
        else:
            govs_d_user_df = pd.concat([govs_d_user_df, my_userdata_df], ignore_index=True)                

        
    except:
        print(f'    ERROR: Could not retrieve Tweepy User Object')
    

    i += 1



In [ ]:
govs_d_user_df.head()

In [ ]:
# 

# Specify the usernames that you want to lookup below
# You can enter up to 100 comma-separated values.
usernames = "usernames=TwitterDev,TwitterAPI"
user_fields = "user.fields=description,created_at"
# User fields are adjustable, options include:
# created_at, description, entities, id, location, name,
# pinned_tweet_id, profile_image_url, protected,
# public_metrics, url, username, verified, and withheld
url = "https://api.twitter.com/2/users/by?{}&{}".format(usernames, user_fields)
    
json_response = connect_to_endpoint_wo_params(url)
print(json.dumps(json_response, indent=4, sort_keys=True))

### User Followers (run once for each gov)

Rate Limit: ? Per App/? Per User (within each 15min window)

(see https://developer.twitter.com/en/docs/twitter-api/rate-limits)

In [ ]:
%whos dict

In [ ]:
# Test
user_id = 2244994945

url = "https://api.twitter.com/2/users/{}/followers".format(user_id)
params = {"user.fields": "created_at"}
json_response = connect_to_endpoint_with_params(url, params)

followers_df = pd.json_normalize(json_response['data'])
followers_ct = followers_df.shape[0]
followers_df.head()
followers_df.info()

In [ ]:
def get_followers(user_id):
    url = "https://api.twitter.com/2/users/{}/followers".format(user_id)
    params = {"user.fields": "created_at"}
    json_response = connect_to_endpoint_with_params(url, params)
    
    followers_df = pd.json_normalize(json_response['data'])
    followers_ct = followers_df.shape[0]

    try:
        page_next_token = json_response['meta']['next_token']
    except:
        pass
    
    if not (page_next_token is None):
        
        while not (page_next_token is None):
            try:
                params = {"user.fields": "created_at", "pagination_token": page_next_token}
                json_response = connect_to_endpoint_with_params(url, params)
                my_followers_df = pd.json_normalize(json_response['data'])
                
                followers_df = pd.concat([followers_df, my_followers_df], ignore_index=True)
                followers_ct += len(json_response['data'])
                print(f'Current users_ct: {followers_ct}')
                
                try:
                    page_next_token = json_response['meta']['next_token']
                except:
                    pass
                      
            except:
                page_next_token = None
    
    print(f'Final users_ct: {followers_df}')
    
    return followers_df

In [ ]:
user_id = 2244994945

govs_d_followers_df = get_followers(user_id)
govs_d_followers_df.head()
govs_d_followers_df.info()

In [ ]:
# Check not just repeating every 100

govs_d_followers_df.iloc[0:10]
govs_d_followers_df.iloc[95:105]

In [ ]:
govs_d_followers_df.tail()

In [ ]:
# Insert user_id as 'destination' in first column
#        rename following id as 'source'

govs_d_followers_df.insert(0, 'destination', user_id)
govs_d_followers_df.rename(columns={'id':'source'}, inplace=True)

In [ ]:
# Verify

govs_d_followers_df.head()
govs_d_followers_df.info()

In [ ]:
# Save

filename_out = f'twitter_follower_{user_id}.csv'
govs_d_followers_df.to_csv(filename_out, index=False)

### Combine All Followers *.csv Files

In [ ]:
datafile_csv_ls = glob.glob('twitter_follower_*.csv')

all_df = pd.DataFrame()

for i, adatafile in enumerate(datafile_csv_ls):
    print(f'Processing #{i}: {adatafile}')
    current_df = pd.read_csv(adatafile, index_col=[0])
    print(f'      shape: {current_df.shape}')
    if i == 0:
        all_df = current_df.copy(deep=True)
    else:
        all_df = pd.concat([all_df, current_df], ignore_index=True)
        
print(f'Combined DataFrame.shape: {all_df.shape}')
    

In [ ]:
# Save Combined DataFrame

filename_out = f'twitter_all_follower_{user_id}.csv'
all_df.to_csv(filename_out, index=False)

### User Following (run once for each gov)

Rate Limit: 15 Per App/15 Per User (within each 15min window)

(see https://developer.twitter.com/en/docs/twitter-api/rate-limits)

In [ ]:
def get_following(user_id):
    url = "https://api.twitter.com/2/users/{}/following".format(user_id)
    params = {"user.fields": "created_at"}
    json_response = connect_to_endpoint_with_params(url, params)
    
    following_df = pd.json_normalize(json_response['data'])
    following_ct = following_df.shape[0]

    try:
        page_next_token = json_response['meta']['next_token']
    except:
        pass
    
    if not (page_next_token is None):
        
        while not (page_next_token is None):
            try:
                params = {"user.fields": "created_at", "pagination_token": page_next_token}
                json_response = connect_to_endpoint_with_params(url, params)
                my_following_df = pd.json_normalize(json_response['data'])
                
                following_df = pd.concat([following_df, my_following_df], ignore_index=True)
                following_ct += len(json_response['data'])
                print(f'Current users_ct: {following_ct}')
                
                try:
                    page_next_token = json_response['meta']['next_token']
                except:
                    pass
                      
            except:
                page_next_token = None
    
    print(f'Final users_ct: {following_ct}')
    
    return following_df

In [ ]:
user_id = 2244994945
govs_d_following_df = get_following(user_id)
govs_d_following_df.head()
govs_d_following_df.info()

In [ ]:
# Check not just repeating every 100

govs_d_following_df.iloc[0:10]
govs_d_following_df.iloc[95:105]

In [ ]:
govs_d_following_df.tail()

In [ ]:
# Insert user_id as 'source' in first column
#        rename following id as 'destination'

govs_d_following_df.insert(0, 'source', user_id)
govs_d_following_df.rename(columns={'id':'destination'}, inplace=True)

In [ ]:
# Verify
 
govs_d_following_df.head()
govs_d_following_df.info()

In [ ]:
# Save

filename_out = f'twitter_following_{user_id}.csv'
govs_d_following_df.to_csv(filename_out, index=False)

### Combine All Following *.csv Files

In [ ]:
datafile_csv_ls = glob.glob('twitter_following_*.csv')

all_df = pd.DataFrame()

for i, adatafile in enumerate(datafile_csv_ls):
    print(f'Processing #{i}: {adatafile}')
    current_df = pd.read_csv(adatafile, index_col=[0])
    print(f'      shape: {current_df.shape}')
    if i == 0:
        all_df = current_df.copy(deep=True)
    else:
        all_df = pd.concat([all_df, current_df], ignore_index=True)
        
print(f'Combined DataFrame.shape: {all_df.shape}')
    

In [ ]:
# Save Combined DataFrame

filename_out = f'twitter_all_following_{user_id}.csv'
all_df.to_csv(filename_out, index=False)

### User Timeline

In [ ]:
def get_user_timeline(id, params):
    url = "https://api.twitter.com/2/users/{}/tweets".format(id)
    # params = {"tweet.fields": "created_at"}
    json_response = connect_to_endpoint_with_params(url, params)
    return json_response


# Tweet fields are adjustable.
# Options include:
# attachments, author_id, context_annotations,
# conversation_id, created_at, entities, geo, id,
# in_reply_to_user_id, lang, non_public_metrics, organic_metrics,
# possibly_sensitive, promoted_metrics, public_metrics, referenced_tweets,
# source, text, and withheld
my_params = {"tweet.fields": "created_at"}

# Test
user_id = 2244994945

json_response = get_user_timeline(user_id, my_params)
print(json.dumps(json_response, indent=4, sort_keys=True))

my_timeline_df = pd.DataFrame(json_response['data'])
my_timeline_df.head()
my_timeline_df.info()

### Liked Tweets

In [ ]:
def get_liked_tweets(id, params):
    url = "https://api.twitter.com/2/users/{}/liked_tweets".format(id)
    # params = {"tweet.fields": "created_at"}
    json_response = connect_to_endpoint_with_params(url, params)
    return json_response


# Tweet fields are adjustable.
# Options include:
# attachments, author_id, context_annotations,
# conversation_id, created_at, entities, geo, id,
# in_reply_to_user_id, lang, non_public_metrics, organic_metrics,
# possibly_sensitive, promoted_metrics, public_metrics, referenced_tweets,
# source, text, and withheld
tweet_fields = "tweet.fields=lang,author_id"
# Be sure to replace your-user-id with your own user ID or one of an authenticating user
# You can find a user ID by using the user lookup endpoint
id = "your-user-id"
# You can adjust ids to include a single Tweets.
# Or you can add to up to 100 comma-separated IDs
url = "https://api.twitter.com/2/users/{}/liked_tweets".format(id)

# Test
user_id = 2244994945

json_response = get_liked_tweets(user_id, tweet_fields)
print(json.dumps(json_response, indent=4, sort_keys=True))

my_liked_tweets_df = pd.DataFrame(json_response['data'])
my_liked_tweets_df.head()
my_liked_tweets_df.info() 

### Liking Users

In [ ]:
json_response = get_liking_users(user_id, user_fields)
pprint(json_response)

In [ ]:
def get_liking_users(id, params):
    url = "https://api.twitter.com/2/tweets/{}/liking_users".format(id)
    # params = {"tweet.fields": "created_at"}
    json_response = connect_to_endpoint_with_params(url, params)
    return json_response


# User fields are adjustable, options include:
# created_at, description, entities, id, location, name,
# pinned_tweet_id, profile_image_url, protected,
# public_metrics, url, username, verified, and withheld
user_fields = "user.fields=created_at,description"
# You can replace the ID given with the Tweet ID you wish to like.
# You can find an ID by using the Tweet lookup endpoint
id = "1590693373056122880"
# You can adjust ids to include a single Tweets.
# Or you can add to up to 100 comma-separated IDs

# Test
user_id = "1590693373056122880"

json_response = get_liking_users(user_id, user_fields)
print(json.dumps(json_response, indent=4, sort_keys=True))

my_liking_users_df = pd.DataFrame(json_response['data'])
my_liking_users_df.head()
my_liking_users_df.info() 

# END OF NOTEBOOK

## Appendix: Set Union

In [ ]:
# Find common elements in 2 data collections (here lists)
# 
aoc_ls = [1,2,3,4,5,6,7,8,9,10]
mgt_ls = [8,9,10,11,12,13,14,15,16,17,18]

common_ls = list(set(aoc_ls).intersection(set(mgt_ls)))

In [ ]:
for auser in common_ls:
    user (user_fields=['created_at','public_metrics'])

## Appendix: Join csv

In [ ]:
aoc_followers_df = pd.read_csv('aoc_following_ids.csv', index_col=[0])
aoc_followers_df.head()
aoc_followers_df.info()

In [ ]:
mtg_followers_df = pd.read_csv('mtg_following_ids.csv', index_col=[0])
mtg_followers_df.head()
mtg_followers_df.info()

In [ ]:
aoc_followers_ls = aoc_followers_df.id.to_list()
len(aoc_followers_ls)
aoc_followers_ls[:5]


In [ ]:
mtg_followers_ls = mtg_followers_df.id.to_list()
len(mtg_followers_ls)
mtg_followers_ls[:5]

In [ ]:
aoc_mtg_followers_ls = list(set(aoc_followers_ls).intersection(set(mtg_followers_ls)))
len(aoc_mtg_followers_ls)
aoc_mtg_followers_ls

## Appendix: Junk

In [ ]:
# Twitter Standard API v1.1
# Get Tweet Timelines
# https://developer.twitter.com/en/docs/twitter-api/v1/tweets/timelines/api-reference/get-statuses-user_timeline

api.user_timeline('GovEvers')

In [ ]:
govs_tweets_df = pd.DataFrame()

for i, agov_username in enumerate(govs_username_ls[:3]):
    print(f'Processing #{i}: {agov_username}')
    get_user_tweets(x.Twitter_username, x.Political_party),axis=1)
gc.collect()
    # govs_tweets_df.append(agov_username)
    # govs_df.apply(lambda x:get_user_tweets(x.Twitter_username, x.Political_party),axis=1)
    df.apply(lambda x:get_user_tweets(x.Twitter_username, x.Political_party),axis=1)
gc.collect()
    gc.collect()